In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [9]:
!rm -rf /kaggle/working/EmoLLM
!git clone https://github.com/SmartFlowAI/EmoLLM.git        

Cloning into 'EmoLLM'...
remote: Enumerating objects: 2016, done.
remote: Counting objects: 100% (532/532), done.
remote: Compressing objects: 100% (258/258), done.
remote: Total 2016 (delta 373), reused 340 (delta 274), pack-reused 1484
Receiving objects: 100% (2016/2016), 86.82 MiB | 27.39 MiB/s, done.
Resolving deltas: 100% (1087/1087), done.
Updating files: 100% (215/215), done.


In [5]:
!pip install -U 'xtuner[deepspeed]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 99.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 90.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [7]:
!pip show xtuner

Name: xtuner
Version: 0.1.17
Summary: An efficient, flexible and full-featured toolkit for fine-tuning large models
Home-page: https://github.com/InternLM/xtuner
Author: XTuner Contributors
Author-email: openmmlab@gmail.com
License: Apache License 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: bitsandbytes, datasets, einops, lagent, mmengine, openpyxl, peft, scikit-image, scipy, SentencePiece, tiktoken, torch, torchvision, transformers, transformers-stream-generator
Required-by: 


In [ ]:
#!huggingface-cli download --resume-download internlm/internlm2-chat-7b --local-dir /root/share/model_repos/internlm2-chat-7b

## Qwen 0.5 full 
-**batch=4,max_length=512 , GPU=14.5G**  
最大内存 = batch * max_length 
例如： 4 * 512 = 1 * 2048


In [8]:
!huggingface-cli download --resume-download Qwen/Qwen1.5-0.5B-Chat --local-dir /kaggle/working/Qwen
config_file = "/kaggle/working/EmoLLM/xtuner_config/qwen1_5_0_5_B_full.py"

# # 首先读取文件内容
# with open(config_file, 'r') as file:
#     content = file.read()

# # 修改config.ini的内容 ，转换bfloat16为float16   
# content = content.replace('./data_pro.json', '/kaggle/working/EmoLLM/datasets/data_pro.json')
# content = content.replace('/root/model/qwen/Qwen1___5-0___5B-Chat','/kaggle/working/Qwen')
# content = content.replace('load_from = \'/root/Emollm/work_dirs/qwen_0_5_B/iter_255.pth\'','load_from = None')

# content = content.replace('batch_size = 4  # per_device','batch_size = 1  # per_device')
# content = content.replace('max_length = 1024','max_length = 512')

# print(content)
# # # 写入新内容
# with open(config_file, 'w') as file:
#     file.write(content)

# 打开并读取config.txt文件
with open(config_file, 'r') as file:
    lines = file.readlines()

lines[22] = 'pretrained_model_name_or_path = "/kaggle/working/Qwen"\n'
lines[26] = 'data_path = "/kaggle/working/EmoLLM/datasets/data_pro.json"\n'  # 注意索引是从0开始
lines[28] = 'max_length = 2048\n'  # 注意索引是从0开始
lines[32] = 'batch_size = 1  # per_device\n'  # 注意索引是从0开始
lines[33] = 'accumulative_counts = 64\n'
lines[182] = 'load_from = None\n'

# 回写到config.txt文件
with open(config_file, 'w') as file:
    file.writelines(lines)
    
with open(config_file, 'r') as file:
     content = file.read()    
print(content)        

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
Fetching 10 files:   0%|                                 | 0/10 [00:00<?, ?it/s]downloading https://huggingface.co/Qwen/Qwen1.5-0.5B-Chat/resolve/f82bd3692de0283f4a4b31e06d164dd8467fb52e/config.json to /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-0.5B-Chat/blobs/f0041555808f66af0491fa912882f59ed29ed414.incomplete
downloading https://huggingface.co/Qwen/Qwen1.5-0.5B-Chat/resolve/f82bd3692de0283f4a4b31e06d164dd8467fb52e/tokenizer.json to /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-0.5B-Chat/blobs/33ea6c72ebb92a237fa2bdf26c5ff16592efcdae.incomplete
downloading https://huggingface.co/Qwen/Qwen1.5-0.5B-Chat/resolve/f82bd3692de0283f4a4b31e06d164dd8467fb52e/.gitattributes to /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-0.5B-Chat/blobs/a6344aac8c09253b3b630fb776ae94478aa0275b.incomplete
downloading https:/

In [ ]:
!NPROC_PER_NODE=2 xtuner train /kaggle/working/EmoLLM/xtuner_config/qwen1_5_0_5_B_full.py --deepspeed deepspeed_zero2

04/13 10:52:05 - mmengine - WARNING - Use random port: 21143
[2024-04-13 10:52:07,241] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-04-13 10:52:11,559] torch.distributed.run: [WARNING] 
[2024-04-13 10:52:11,559] torch.distributed.run: [WARNING] *****************************************
[2024-04-13 10:52:11,559] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-04-13 10:52:11,559] torch.distributed.run: [WARNING] *****************************************
[2024-04-13 10:52:15,632] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-04-13 10:52:15,660] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/mmengine

# Internlm2 1.8b Chat


In [10]:
!huggingface-cli download --resume-download internlm/internlm2-chat-1_8b --local-dir /kaggle/working/internlm2
config_file = "/kaggle/working/EmoLLM/xtuner_config/internlm2_7b_chat_qlora_e3.py"


# 打开并读取config.txt文件
with open(config_file, 'r') as file:
    lines = file.readlines()

lines[23] = 'pretrained_model_name_or_path = "/kaggle/working/internlm2"\n'
lines[27] = 'data_path = "/kaggle/working/EmoLLM/datasets/data_pro.json"\n'  # 注意索引是从0开始
lines[29] = 'max_length = 2048\n'  # 注意索引是从0开始
lines[33] = 'batch_size = 1  # per_device\n'  # 注意索引是从0开始
lines[34] = 'accumulative_counts = 16\n'
# lines[182] = 'load_from = None\n'

# 回写到config.txt文件
with open(config_file, 'w') as file:
    file.writelines(lines)
    
# with open(config_file, 'r') as file:
#      content = file.read()    
# print(content)        

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
Fetching 14 files:   0%|                                 | 0/14 [00:00<?, ?it/s]downloading https://huggingface.co/internlm/internlm2-chat-1_8b/resolve/ecccbb5c87079ad84e5788baa55dd6e21a9c614d/generation_config.json to /root/.cache/huggingface/hub/models--internlm--internlm2-chat-1_8b/blobs/4b388c5f8fa584ebd865a2e9a6e5ced521a42646.incomplete
downloading https://huggingface.co/internlm/internlm2-chat-1_8b/resolve/ecccbb5c87079ad84e5788baa55dd6e21a9c614d/model.safetensors.index.json to /root/.cache/huggingface/hub/models--internlm--internlm2-chat-1_8b/blobs/e6c64e5f675cd66006d652b89a13230276991efa.incomplete
downloading https://huggingface.co/internlm/internlm2-chat-1_8b/resolve/ecccbb5c87079ad84e5788baa55dd6e21a9c614d/README.md to /root/.cache/huggingface/hub/models--internlm--internlm2-chat-1_8b/blobs/d5c1028274d46f2

In [1]:
!pip install lmdeploy==0.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 MB 21.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 92.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 9.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.8/451.8 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 8.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━

In [ ]:
!NPROC_PER_NODE=2 xtuner train /kaggle/working/EmoLLM/xtuner_config/internlm2_7b_chat_qlora_e3.py --deepspeed deepspeed_zero2

04/13 12:00:47 - mmengine - WARNING - Use random port: 22119
[2024-04-13 12:00:50,090] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-04-13 12:00:54,413] torch.distributed.run: [WARNING] 
[2024-04-13 12:00:54,413] torch.distributed.run: [WARNING] *****************************************
[2024-04-13 12:00:54,413] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-04-13 12:00:54,413] torch.distributed.run: [WARNING] *****************************************
[2024-04-13 12:00:58,897] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-04-13 12:00:59,235] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/mmengine